import pandas as pd
df=pd.read_csv('creditcard.csv')
df.head()

In [2]:
df.shape

(284807, 31)

In [3]:
df.isnull().sum() # no null values

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [4]:
df['Class'].value_counts()  # fraud detection based on class so we take value counts

# as there is huge difference it is imbalanced dataset

0    284315
1       492
Name: Class, dtype: int64

In [5]:
# Segregating X and Y to Independent and dependent features

X=df.drop('Class',axis=1)
y=df.Class

### CrossValidation like KFold and HyperparameterTuning

Imbalanced dataset does not heavely impact ensembled techniques or decission trees are actually used.

During imbalanced dataset donot check just the accuracy

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import KFold
import numpy as np #to apply kfold
from sklearn.model_selection import GridSearchCV #CrossValidation

In [7]:
log_class=LogisticRegression()
# crossvalidation
grid={'C':10.0**np.arange(-2,3),'penalty':['l1','l2']}# 'C' is hyperparameter
# grid is giving some list of values for performing hyperparameter tuning
# penalty key is just like regularization 

cv=KFold(n_splits=5,random_state=None,shuffle=False)  # applying crossvalidation for kfold splits as 5

In [8]:
# We perform train test split before Grid SearchCV
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.7)

In [9]:
clf=GridSearchCV(log_class,grid,cv=cv,n_jobs=-1,scoring='f1_macro')# f1_macro is scoring parameter we can also use accuracy 
clf.fit(X_train,y_train)             #n_jobs uses all the cores of the computer


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
Valu

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro')

In [10]:
y_pred=clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
# if for imbalanced dataset if we get better accuracy someting is wrong

# we think to increase the precission and recall values 
#and to reduce the false +ve and -ve values

[[85249    47]
 [   43   104]]
0.9989466661985184
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.69      0.71      0.70       147

    accuracy                           1.00     85443
   macro avg       0.84      0.85      0.85     85443
weighted avg       1.00      1.00      1.00     85443



In [11]:
# we do this Random Forest in the last
# RanfomForest is better than all the techiques better than under/OverSampling smote etc...
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier() # decisssion tree plays an importantent role it makes a hierarchicle structure
classifier.fit(X_train,y_train)

# to increase or decrease False+ve and _ve values we can make use of class_weight sattribute

RandomForestClassifier()

In [13]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85286    10]
 [   31   116]]
0.9995201479348805
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.92      0.79      0.85       147

    accuracy                           1.00     85443
   macro avg       0.96      0.89      0.92     85443
weighted avg       1.00      1.00      1.00     85443



In [17]:
# increaseing th class weights
y_train.value_counts()
 # here for each 1's in the class we increase weight

0    199019
1       345
Name: Class, dtype: int64

In [18]:
# to increase the class weights we need dictionaries
class_weight=dict({0:1,1:100})

# this implies for 0's increase by 0 times, for 1's increase by 100 times

In [19]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier(class_weight=class_weight) # decisssion tree plays an importantent role it makes a hierarchicle structure
classifier.fit(X_train,y_train)

RandomForestClassifier(class_weight={0: 1, 1: 100})

In [20]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85289     7]
 [   34   113]]
0.9995201479348805
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.94      0.77      0.85       147

    accuracy                           1.00     85443
   macro avg       0.97      0.88      0.92     85443
weighted avg       1.00      1.00      1.00     85443



### Undersampling 

we try to reduce the points of maximum labels/class values

By doing this we loose some data (apply only for lessdataset)

In [21]:
y_train.value_counts()

# we reduce 80% of the 0's class

0    199019
1       345
Name: Class, dtype: int64

In [22]:
from collections import Counter# counts the no of 0's and 1's

In [33]:
from imblearn.under_sampling import NearMiss
ns=NearMiss()
X_train_ns,y_train_ns=ns.fit_resample(X_train,y_train)
print("No of classer before fit{}".format(Counter(y_train)))
print("No of classer before fit{}".format(Counter(y_train_ns)))

No of classer before fitCounter({0: 199019, 1: 345})
No of classer before fitCounter({0: 345, 1: 345})


In [34]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train_ns,y_train_ns)

RandomForestClassifier()

In [35]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

# presision is gone many errors in test dataset[74348 10948]


[[74348 10948]
 [    6   141]]
0.8717975726507731
              precision    recall  f1-score   support

           0       1.00      0.87      0.93     85296
           1       0.01      0.96      0.03       147

    accuracy                           0.87     85443
   macro avg       0.51      0.92      0.48     85443
weighted avg       1.00      0.87      0.93     85443



### OverSampling

We imput and increase the values having the lowernumber of values

It is inverse of UnderSampling

In [36]:
from imblearn.over_sampling import RandomOverSampler

In [38]:
os=RandomOverSampler()
X_train_os,y_train_os=os.fit_resample(X_train,y_train)
print("No of classer before fit{}".format(Counter(y_train)))
print("No of classer before fit{}".format(Counter(y_train_os)))

No of classer before fitCounter({0: 199019, 1: 345})
No of classer before fitCounter({1: 199019, 0: 199019})


In [39]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train_os,y_train_os)

RandomForestClassifier()

In [40]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

#false -ve is redused

[[85290     6]
 [   30   117]]
0.9995786664794073
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.95      0.80      0.87       147

    accuracy                           1.00     85443
   macro avg       0.98      0.90      0.93     85443
weighted avg       1.00      1.00      1.00     85443



#### SMOTE Tomek

It creates more/new points of the lowest numbers

It takes more time w.r.t oversampling

It is the combinatio of both undersampling and oversampling

In [41]:
from imblearn.combine import SMOTETomek

In [ ]:
smo=SMOTETomek()
X_train_smo,y_train_smo=smo.fit_resample(X_train,y_train)
print("No of classer before fit{}".format(Counter(y_train)))
print("No of classer before fit{}".format(Counter(y_train_smo)))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train_smo,y_train_smo)

In [ ]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

### Ensemble Techniques



In [ ]:
from imblearn.ensemble import EasyEnsembleClassifier

In [ ]:
easy=EasyEnsembleClassifier()
easy.fit(X_train,y_train)

In [ ]:
y_pred=easy.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

if dataset small ill go with undersample focus on performance matrix like : accurace,precession,recall F1-score

Apart from that based on domain knowledge weathere we should reduce false +ve _ve based on that I'll be considering my ROC score(i.e Probability value) 

Finally smote, oversampling by involving around the understanding performance matrix

Finally if above is Not Working, I'll go with ensemblled techniques like decission trees, random forestxgboost where i can provid class width parameters

Ensembled techniques prevents imbalence datasset

We apply sampling techniques during model creation coz we need to validate/verify it